In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import uproot
import awkward as ak
import pandas as pd

from tqdm.notebook import tqdm

import time

import pickle


# Loading POTs

In [ ]:
# from Sergey, Slack, 2024_10_18
loc = "BNBNuMI_TOsc_input_numifluxpatched/hist_rootfiles/"

pot_dic = {}

beams = [
    "BNB", 
    "NuMI",
]

runs = [
    "run1",
    "run2",
    "run3",
]

horns = [
    "FHC",
    "RHC",
]


filetypes = [
    "data",
    "ext",
    "dirt",
    "nu_overlay", 
    "intrinsic_nue",
    "fullosc_overlay",
]

for filetype in filetypes:
    for beam in beams:
        for run in runs:
            for horn in horns:

                if beam == "BNB":
                    if horn == "RHC":
                        continue # no BNB RHC files
                    runhorn = run
                else:
                    runhorn = run + "_" + horn
                if runhorn == "run3_FHC":
                    continue

                if filetype == "data":
                    filename = f"{runhorn}_data_{beam}.root"
                elif filetype == "ext":
                    filename = f"checkout_data_ext{beam}_{runhorn}.root"
                elif filetype == "dirt" and beam == "BNB":
                    filename = f"checkout_prodgenie_dirt_overlay_{runhorn}.root"
                elif filetype == "dirt" and beam == "NuMI":
                    filename = f"checkout_prodgenie_{runhorn.lower()}_dirt.root"
                elif filetype == "nu_overlay" and beam == "BNB":
                    filename = f"checkout_prodgenie_bnb_nu_overlay_{runhorn.lower()}.root"
                elif filetype == "nu_overlay" and beam == "NuMI":
                    filename = f"checkout_prodgenie_{runhorn.lower()}_nu_overlay.root"
                elif filetype == "intrinsic_nue" and beam == "BNB":
                    filename = f"checkout_prodgenie_bnb_intrinsic_nue_overlay_{runhorn.lower()}.root"
                elif filetype == "intrinsic_nue" and beam == "NuMI":
                    filename = f"checkout_prodgenie_{runhorn.lower()}_intrinsic_nue_overlay.root"
                elif filetype == "fullosc_overlay" and beam == "BNB":
                    filename = f"checkout_prodgenie_bnb_numu2nue_overlay_{runhorn.lower()}.root"
                elif filetype == "fullosc_overlay" and beam == "NuMI":
                    filename = f"checkout_prodgenie_{runhorn.lower()}_fullosc_overlay.root"
                else:
                    raise ValueError(f"Invalid filetype: {filetype} for beam: {beam} and runhorn: {runhorn}")

                f = uproot.open(loc+filename)

                pot_dic[f"{beam}_{runhorn}_{filetype}"] = f["T"]["pot"].array()[0]

for k, v in pot_dic.items():
    print(k, v)


# Loading roofile things into dataframe (no osc samples)

In [ ]:
beams = [
    "BNB", 
    "NuMI",
]

runs = [
    "run1",
    "run2",
    "run3",
]

horns = [
    "FHC",
    "RHC",
]

sels = [
    ("nueCC", "_FC"),
    ("nueCC", "_PC"),
    ("numuCC", "_FC"),
    ("numuCC", "_PC"),
    ("CCpi0", "_FC"),
    ("CCpi0", "_PC"),
    ("NCpi0", ""),
]

filetypes = [
    ("intrinsic", "intnue"), 
    ("nu_overlay", "overlaynumu"),
    ("appnue", "appnue"),
]

dfs = []

for beam in beams:
    for filetype_pair in filetypes:
        for run in runs:
            for horn in horns:

                if beam == "BNB":
                    if horn == "RHC":
                        continue # no BNB RHC files
                    runhorn = run
                else:
                    runhorn = run + "_" + horn

                if runhorn == "run3_FHC":
                    continue

                filename = f"roofile_obj_{beam}_{runhorn}_{filetype_pair[0]}.root"
                f = uproot.open(loc+filename)

                for sel in sels:

                    try:
                        key = f"tree_{sel[0]}_from_{filetype_pair[1]}{sel[1]}"
                        curr_df = pd.DataFrame()
                        for subkey in f[key].keys():
                            curr_df[subkey] = f[key][subkey].array(library="pd")

                        curr_df["beam"] = beam
                        curr_df["filetype"] = filetype_pair[0]
                        curr_df["runhorn"] = runhorn
                        curr_df["sel"] = sel[0] + sel[1]

                        if filetype_pair[0] == "intrinsic":
                            curr_df["pot_weight"] = pot_dic[f"{beam}_{runhorn}_data"] / pot_dic[f"{beam}_{runhorn}_intrinsic_nue"]
                        else:
                            curr_df["pot_weight"] = pot_dic[f"{beam}_{runhorn}_data"] / pot_dic[f"{beam}_{runhorn}_nu_overlay"]
                        dfs.append(curr_df)
                    except KeyError:
                        pass

all_sig_df = pd.concat(dfs, ignore_index=True, axis=0)


energy_bins = np.concatenate([np.linspace(0, 2500, 26), [1e9]])
energy_bin_centers = (energy_bins[:-1] + energy_bins[1:]) / 2
energy_bin_centers[-1] = 2550

all_sig_df["non_osc_weight"] = all_sig_df["pot_weight"] * all_sig_df["e2e_weight_xs"]

all_sig_df = all_sig_df[["beam", "runhorn", "filetype", "sel", "e2e_pdg", "e2e_Etrue", "e2e_Ereco", "e2e_weight_xs", "e2e_baseline", "non_osc_weight"]]

all_sig_df["osc_weight"] = 1
all_sig_df.loc[all_sig_df["filetype"] == "appnue", "osc_weight"] = 0

all_sig_df["Ereco_bin_index"] = np.digitize(all_sig_df["e2e_Ereco"], bins=energy_bins) - 1

all_sig_df["beam_offset"] = [0 if beam == "BNB" else 26*7 for beam in all_sig_df["beam"]]
sels = all_sig_df["sel"].to_numpy()
sel_offsets = []
for sel in sels:
    if sel == "nueCC_FC":
        sel_offsets.append(0)
    elif sel == "nueCC_PC":
        sel_offsets.append(1)
    elif sel == "numuCC_FC":
        sel_offsets.append(2)
    elif sel == "numuCC_PC":
        sel_offsets.append(3)
    elif sel == "CCpi0_FC":
        sel_offsets.append(4)
    elif sel == "CCpi0_PC":
        sel_offsets.append(5)
    elif sel == "NCpi0":
        sel_offsets.append(6)
    else:
        raise ValueError(f"Invalid sel: {sel}!")
all_sig_df["sel_offset"] = sel_offsets

all_sig_df["reco_bin"] = all_sig_df["beam_offset"] + all_sig_df["sel_offset"] * 26 + all_sig_df["Ereco_bin_index"]
all_sig_df

In [ ]:
nue_disapp_df = all_sig_df.query(f"filetype=='intrinsic' or (filetype=='nu_overlay' and abs(e2e_pdg)==12)").sort_values(by="reco_bin")
numu_disapp_df = all_sig_df.query(f"filetype=='nu_overlay' and abs(e2e_pdg)==14").sort_values(by="reco_bin")
nue_to_numu_df = all_sig_df.query(f"filetype=='appnue' and e2e_pdg==12").sort_values(by="reco_bin")
antinue_to_antinumu_df = all_sig_df.query(f"filetype=='appnue' and e2e_pdg==-12").sort_values(by="reco_bin")

nue_disapp_arr = nue_disapp_df[["e2e_Etrue", "e2e_baseline", "non_osc_weight", "osc_weight"]].to_numpy()
numu_disapp_arr = numu_disapp_df[["e2e_Etrue", "e2e_baseline", "non_osc_weight", "osc_weight"]].to_numpy()
numu_to_nue_arr = nue_to_numu_df[["e2e_Etrue", "e2e_baseline", "non_osc_weight", "osc_weight"]].to_numpy()
antinumu_to_antinue_arr = antinue_to_antinumu_df[["e2e_Etrue", "e2e_baseline", "non_osc_weight", "osc_weight"]].to_numpy()

nue_disapp_reco_idx = np.squeeze(nue_disapp_df[["reco_bin"]].to_numpy())
numu_disapp_reco_idx = np.squeeze(numu_disapp_df[["reco_bin"]].to_numpy())
numu_to_nue_reco_idx = np.squeeze(nue_to_numu_df[["reco_bin"]].to_numpy())
antinumu_to_antinue_reco_idx = np.squeeze(antinue_to_antinumu_df[["reco_bin"]].to_numpy())

sig_arrs = [nue_disapp_arr, numu_disapp_arr, numu_to_nue_arr, antinumu_to_antinue_arr]


# Loading data and bkg hist_rootfiles into dataframe

In [ ]:
data_bkg_hists = {}

beams = [
    "BNB", 
    "NuMI",
]

runs = [
    "run1",
    "run2",
    "run3",
]

horns = [
    "FHC",
    "RHC",
]

filetypes = [
    "data",
    "ext",
    "dirt",
]

for filetype in filetypes:
    for beam in beams:
        for run in runs:
            for horn in horns:

                if beam == "BNB":
                    if horn == "RHC":
                        continue # no BNB RHC files
                    runhorn = run
                else:
                    runhorn = run + "_" + horn
                if runhorn == "run3_FHC":
                    continue

                if filetype == "data":
                    filename = f"{runhorn}_data_{beam}.root"
                elif filetype == "ext":
                    filename = f"checkout_data_ext{beam}_{runhorn}.root"
                elif filetype == "dirt":
                    if beam == "BNB":
                        filename = f"checkout_prodgenie_dirt_overlay_{runhorn}.root"
                    else:
                        filename = f"checkout_prodgenie_{runhorn.lower()}_dirt.root"

                f = uproot.open(loc+filename)
                keys = f.keys()[:]

                if filetype == "data":
                    start_list = [
                        "nueCC_FC",
                        "nueCC_PC",
                        "numuCC_nopi0_nonueCC_FC",
                        "numuCC_nopi0_nonueCC_PC",
                        "CCpi0_nonueCC_FC",
                        "CCpi0_nonueCC_PC",
                        "NCpi0_nonueCC",
                    ]

                arr_count = 0
                filtered_keys = []
                for start in start_list:
                    if filetype != "data":
                        start = "BG_" + start
                    for key in keys:
                        if key.startswith(start):
                            filtered_keys.append(key)
                            arr_count += 1
                            break

                assert arr_count == 7, f"Expected to find 7 arrays, found {arr_count}!"
                
                total_arr = np.array([])
                for key in filtered_keys:
                    total_arr = np.concatenate([total_arr, np.array(f[key].values(flow=True)[1:])])
                data_bkg_hists[f"{beam}_{runhorn}_{filetype}"] = total_arr

                
for k, v in data_bkg_hists.items():
    print(k, len(v))


In [ ]:
ext_hist = np.zeros(26*7*2)
dirt_hist = np.zeros(26*7*2)
data_hist = np.zeros(26*7*2)

for filetype in filetypes:
    curr_arr = np.zeros(26*7*2)
    for beam in beams:
        for run in runs:
            for horn in horns:
                if beam == "BNB":
                    if horn == "RHC":
                        continue # no BNB RHC files
                    runhorn = run
                else:
                    runhorn = run + "_" + horn
                if runhorn == "run3_FHC":
                    continue

                if beam == "BNB":
                    curr_arr[:26*7] += data_bkg_hists[f"{beam}_{runhorn}_{filetype}"]
                else:
                    curr_arr[26*7:] += data_bkg_hists[f"{beam}_{runhorn}_{filetype}"]

    if filetype == "ext":
        ext_hist = curr_arr
    elif filetype == "dirt":
        dirt_hist = curr_arr
    elif filetype == "data":
        data_hist = curr_arr


# Loading Systematics

In [ ]:
uncollapsed_flux_frac_cov = np.zeros((26*7*2*3, 26*7*2*3))
uncollapsed_xs_frac_cov = np.zeros((26*7*2*3, 26*7*2*3))
for i in range(1, 18):
    f = uproot.open(loc + f"XsFlux/cov_{i}.root")
    cov = f[f"frac_cov_xf_mat_{i}"].member("fElements").reshape(1092, 1092)
    if i == 17:
        uncollapsed_xs_frac_cov += cov
    else:
        uncollapsed_flux_frac_cov += cov

uncollapsed_detvar_frac_cov_missing_fullosc = np.zeros((26*7*2*3, 26*7*2*3))
cov_names = ["LYDown", "LYRayleigh", "Recomb2", "SCE", "dEdX", "WMThetaXZ", "WMThetaYZ", "WMX", "WMYZ", "LYAtt"]
for i in range(1, 11):
    if i == 5: 
        continue # no deE/dx
    cov_name = cov_names[i-1]
    f = uproot.open(loc + f"DetVar/cov_{cov_name}.root")
    cov = f[f"frac_cov_det_mat_{i}"].member("fElements").reshape(1092, 1092)
    uncollapsed_detvar_frac_cov_missing_fullosc += cov

zeros = np.zeros((26*7, 26*7))
uncollapsed_detvar_frac_cov_bnb_bnb = uncollapsed_detvar_frac_cov_missing_fullosc[:26*7, :26*7]
uncollapsed_detvar_frac_cov_bnb_numi = uncollapsed_detvar_frac_cov_missing_fullosc[:26*7, 26*7*3:26*7*4]
uncollapsed_detvar_frac_cov_numi_numi = uncollapsed_detvar_frac_cov_missing_fullosc[26*7*3:26*7*4, 26*7*3:26*7*4]

# uncollapsed_detvar_frac_cov_bnb_numi should be symmetrical, but saw much better eigenvalue behavior when using .T below

uncollapsed_detvar_frac_cov = np.block([
    [uncollapsed_detvar_frac_cov_bnb_bnb,   zeros,  uncollapsed_detvar_frac_cov_bnb_bnb,    uncollapsed_detvar_frac_cov_bnb_numi,  zeros,  uncollapsed_detvar_frac_cov_bnb_numi],
    [zeros,                                 zeros,  zeros,                                  zeros,                                 zeros,  zeros],
    [uncollapsed_detvar_frac_cov_bnb_bnb,   zeros,  uncollapsed_detvar_frac_cov_bnb_bnb,    uncollapsed_detvar_frac_cov_bnb_numi,  zeros,  uncollapsed_detvar_frac_cov_bnb_numi],
    [uncollapsed_detvar_frac_cov_bnb_numi.T,  zeros,  uncollapsed_detvar_frac_cov_bnb_numi.T,   uncollapsed_detvar_frac_cov_numi_numi, zeros,  uncollapsed_detvar_frac_cov_numi_numi],
    [zeros,                                 zeros,  zeros,                                  zeros,                                 zeros,  zeros],
    [uncollapsed_detvar_frac_cov_bnb_numi.T,  zeros,  uncollapsed_detvar_frac_cov_bnb_numi.T,   uncollapsed_detvar_frac_cov_numi_numi, zeros,  uncollapsed_detvar_frac_cov_numi_numi],
])

uncollapsed_frac_cov = uncollapsed_xs_frac_cov + uncollapsed_flux_frac_cov + uncollapsed_detvar_frac_cov


In [ ]:
# these plots can be compared with https://microboone-docdb.fnal.gov/cgi-bin/sso/ShowDocument?docid=40396

plt.figure(figsize=(10, 6))
plt.plot(np.concatenate([np.sqrt(np.diag(uncollapsed_flux_frac_cov)[:26*7]), np.sqrt(np.diag(uncollapsed_flux_frac_cov)[26*7*2:26*7*3])]), label="Flux", c="r")
plt.plot(np.concatenate([np.sqrt(np.diag(uncollapsed_xs_frac_cov)[:26*7]), np.sqrt(np.diag(uncollapsed_xs_frac_cov)[26*7*2:26*7*3])]), label="Xs", c="b")
plt.plot(np.concatenate([np.sqrt(np.diag(uncollapsed_detvar_frac_cov)[:26*7]), np.sqrt(np.diag(uncollapsed_detvar_frac_cov)[26*7*2:26*7*3])]), label="DetVar", c="fuchsia")
plt.plot(np.concatenate([np.sqrt(np.diag(uncollapsed_frac_cov)[:26*7]), np.sqrt(np.diag(uncollapsed_frac_cov)[26*7*2:26*7*3])]), label="total", c="k", ls="--")
plt.ylim(0, 2.5)
plt.title("BNB mc_pred, BNB fullosc")
plt.legend()
plt.show()

plt.figure(figsize=(10, 6))
plt.plot(np.concatenate([np.sqrt(np.diag(uncollapsed_flux_frac_cov)[26*7*3:26*7*4]), np.sqrt(np.diag(uncollapsed_flux_frac_cov)[26*7*5:26*7*6])]), label="Flux", c="r")
plt.plot(np.concatenate([np.sqrt(np.diag(uncollapsed_xs_frac_cov)[26*7*3:26*7*4]), np.sqrt(np.diag(uncollapsed_xs_frac_cov)[26*7*5:26*7*6])]), label="Xs", c="b")
plt.plot(np.concatenate([np.sqrt(np.diag(uncollapsed_detvar_frac_cov)[26*7*3:26*7*4]), np.sqrt(np.diag(uncollapsed_detvar_frac_cov)[26*7*5:26*7*6])]), label="DetVar", c="fuchsia")
plt.plot(np.concatenate([np.sqrt(np.diag(uncollapsed_frac_cov)[26*7*3:26*7*4]), np.sqrt(np.diag(uncollapsed_frac_cov)[26*7*5:26*7*6])]), label="total", c="k", ls="--")
plt.ylim(0, 2.5)
plt.title("NuMI mc_pred, NuMI fullosc")
plt.legend()
plt.show()


In [ ]:
# changing from (BNB mc_pred, ext, fullosc), (NuMI mc_pred, ext, fullosc)
# to (BNB mc_pred, NuMI mc_pred), (BNB ext, NuMI ext), (BNB fullosc, NuMI fullosc)
bnb_mcpred_start_index = 0
bnb_ext_start_index = 26*7
bnb_fullosc_start_index = 26*7*2
numi_mcpred_start_index = 26*7*3
numi_ext_start_index = 26*7*4
numi_fullosc_start_index = 26*7*5
new_ordering = np.concatenate([
    np.arange(bnb_mcpred_start_index, bnb_mcpred_start_index + 26*7),
    np.arange(numi_mcpred_start_index, numi_mcpred_start_index + 26*7),
    np.arange(bnb_ext_start_index, bnb_ext_start_index + 26*7),
    np.arange(numi_ext_start_index, numi_ext_start_index + 26*7),
    np.arange(bnb_fullosc_start_index, bnb_fullosc_start_index + 26*7),
    np.arange(numi_fullosc_start_index, numi_fullosc_start_index + 26*7),
])

uncollapsed_frac_cov = uncollapsed_frac_cov[new_ordering, :][:, new_ordering]

print("min eigenval of uncollapsed_frac_cov:", np.min(np.linalg.eigvalsh(uncollapsed_frac_cov)))


In [ ]:
plt.figure()
plt.imshow(uncollapsed_frac_cov, norm=mpl.colors.LogNorm())
plt.colorbar()
plt.title("Fractional Covariance Matrix")
plt.show()

frac_errs = np.sqrt(np.diag(uncollapsed_frac_cov))

correlation_matrix = uncollapsed_frac_cov / np.outer(frac_errs, frac_errs)
plt.figure()
plt.imshow(correlation_matrix, vmin=-1, vmax=1, cmap="coolwarm")
plt.colorbar()
plt.title("Correlation Matrix")
plt.show()


In [ ]:
# mc stat log files created here: https://github.com/BNLIF/wcp-uboone-bdt/blob/main/apps/merge_hist.cxx#L243:
# mc stat log files loaded here: https://github.com/BNLIF/wcp-uboone-bdt/blob/67fdfdf635c0872bc07ca141e211b8f2281308b8/src/TLee.cxx#L2790
# read here: https://github.com/BNLIF/wcp-uboone-bdt/blob/67fdfdf635c0872bc07ca141e211b8f2281308b8/src/TLee.cxx#L2819
# gbin>>lbin>>val_pred>>mc_stat>>nn_stat;
# nn_stat doesn't seem to be used
# mc_stat seems to be the covariance, with no sqrt applied


with open("BNBNuMI_TOsc_input_numifluxpatched/0.log", "r") as f:
    text = f.read()

lines = text.split("\n")[1:-1]

pred_stat_diag_cov = []
for line in lines:
    pred_stat_diag_cov.append(float(line.split()[3]))
pred_stat_cov = np.diag(pred_stat_diag_cov)


In [ ]:
dirt_cv_uni = dirt_hist
dirt_var_uni = dirt_hist * 1.5

delta = dirt_var_uni - dirt_cv_uni

extra_dirt_cov = np.outer(delta, delta)

plt.figure()
plt.imshow(extra_dirt_cov, norm=mpl.colors.LogNorm())
plt.colorbar()
plt.show()


# Collapsing Into Histograms

In [ ]:
collapsing_matrix = np.zeros((26*7*2, 26*7*2*3))
for i in range(26*7*2):
    collapsing_matrix[i, i] = 1             # mc_pred
    collapsing_matrix[i, i+26*7*2] = 1        # ext
    collapsing_matrix[i, i+26*7*2*2] = 1      # fullosc


def get_hist_from_arr(reco_idx, arr):
    hist = np.zeros(26*7*2)
    np.add.at(hist, reco_idx, arr[:, 2] * arr[:, 3]) # most of the time is spent in this function! 0.01-0.08 seconds each time, happens 4x per oscillation calculation 
    return hist


def get_hist_cov(sig_arrs):

    nue_disapp_arr, numu_disapp_arr, numu_to_nue_arr, antinumu_to_antinue_arr = sig_arrs

    mc_pred_hist = get_hist_from_arr(nue_disapp_reco_idx, nue_disapp_arr) + get_hist_from_arr(numu_disapp_reco_idx, numu_disapp_arr) + dirt_hist
    fullosc_hist = get_hist_from_arr(numu_to_nue_reco_idx, numu_to_nue_arr) + get_hist_from_arr(antinumu_to_antinue_reco_idx, antinumu_to_antinue_arr)

    uncollapsed_hist = np.concatenate([mc_pred_hist, ext_hist, fullosc_hist])

    uncollapsed_cov = uncollapsed_frac_cov * np.outer(uncollapsed_hist, uncollapsed_hist)

    collapsed_cov = collapsing_matrix @ uncollapsed_cov @ collapsing_matrix.T
    collapsed_hist = collapsing_matrix @ uncollapsed_hist

    cov = collapsed_cov + pred_stat_cov + extra_dirt_cov

    return collapsed_hist, cov


In [ ]:
nue_disapp_arr, numu_disapp_arr, numu_to_nue_arr, antinumu_to_antinue_arr = sig_arrs

mc_pred_hist = get_hist_from_arr(nue_disapp_reco_idx, nue_disapp_arr) + get_hist_from_arr(numu_disapp_reco_idx, numu_disapp_arr) + dirt_hist
fullosc_hist = get_hist_from_arr(numu_to_nue_reco_idx, numu_to_nue_arr) + get_hist_from_arr(antinumu_to_antinue_reco_idx, antinumu_to_antinue_arr)

uncollapsed_hist = np.concatenate([mc_pred_hist, ext_hist, fullosc_hist])

uncollapsed_cov = uncollapsed_frac_cov * np.outer(uncollapsed_hist, uncollapsed_hist)

collapsed_cov = collapsing_matrix @ uncollapsed_cov @ collapsing_matrix.T
collapsed_hist = collapsing_matrix @ uncollapsed_hist

cov = collapsed_cov + pred_stat_cov + extra_dirt_cov

# Nominal Prediction Histogram

In [ ]:
from scipy.special import erfcinv
from scipy.stats import chi2

def get_significance(chisquare, ndf):
    
    # probability of getting a more extreme result
    p_value = 1. - chi2.cdf(chisquare, ndf)
    
    sigma = np.sqrt(2.) * erfcinv(p_value)
    
    return p_value, sigma


In [ ]:
def plot_nue_numu_hists(sig_arrs, no_osc_hist=None):

    hist, cov = get_hist_cov(sig_arrs)

    pred_errs = np.sqrt(np.diag(cov))

    data_errs = np.sqrt(data_hist)

    diffs = data_hist - hist

    chi2 = diffs @ np.linalg.inv(cov + np.diag(hist + 0.001)) @ diffs
    ndf = len(data_hist)

    print(f"Chi2/ndf: {chi2:.2f} / {ndf}, p={get_significance(chi2, ndf)[0]:.8f}, sigma={get_significance(chi2, ndf)[1]:.2f}")

    fig, axs = plt.subplots(2, 4, figsize=(15, 8))

    axs[0, 0].hist(energy_bin_centers, weights=hist[:26], bins=energy_bins, color="b", histtype="step")
    for i in range(26):
        axs[0, 0].add_patch(plt.Rectangle((energy_bins[i], hist[i] - pred_errs[i]), energy_bins[i+1] - energy_bins[i], 2*pred_errs[i], hatch="////", lw=0, fill=False, edgecolor="grey"))
    axs[0, 0].errorbar(energy_bin_centers, data_hist[:26], yerr=data_errs[:26], fmt=".", color="k", label="Data")
    axs[0, 0].set_xlim(0, 2600)
    axs[0, 0].set_title("BNB nueCC FC")
    if no_osc_hist is not None:
        axs[0, 0].hist(energy_bin_centers, weights=no_osc_hist[:26], bins=energy_bins, color="r", histtype="step")

    axs[0, 1].hist(energy_bin_centers, weights=hist[26:26*2], bins=energy_bins, color="b", histtype="step")
    for i in range(26):
        axs[0, 1].add_patch(plt.Rectangle((energy_bins[i], hist[26+i] - pred_errs[26+i]), energy_bins[i+1] - energy_bins[i], 2*pred_errs[26+i], hatch="////", lw=0, fill=False, edgecolor="grey"))
    axs[0, 1].errorbar(energy_bin_centers, data_hist[26:26*2], yerr=data_errs[26:26*2], fmt=".", color="k", label="Data")
    axs[0, 1].set_xlim(0, 2600)
    axs[0, 1].set_title("BNB nueCC PC")
    if no_osc_hist is not None:
        axs[0, 1].hist(energy_bin_centers, weights=no_osc_hist[26:26*2], bins=energy_bins, color="r", histtype="step")

    axs[0, 2].hist(energy_bin_centers, weights=hist[26*2:26*3], bins=energy_bins, color="b", histtype="step")
    for i in range(26):
        axs[0, 2].add_patch(plt.Rectangle((energy_bins[i], hist[26*2+i] - pred_errs[26*2+i]), energy_bins[i+1] - energy_bins[i], 2*pred_errs[26*2+i], hatch="////", lw=0, fill=False, edgecolor="grey"))
    axs[0, 2].errorbar(energy_bin_centers, data_hist[26*2:26*3], yerr=data_errs[26*2:26*3], fmt=".", color="k", label="Data")
    axs[0, 2].set_xlim(0, 2600)
    axs[0, 2].set_title("BNB numuCC FC")
    if no_osc_hist is not None:
        axs[0, 2].hist(energy_bin_centers, weights=no_osc_hist[26*2:26*3], bins=energy_bins, color="r", histtype="step")

    if no_osc_hist is not None:
        axs[0, 3].hist(energy_bin_centers, weights=hist[26*3:26*4], bins=energy_bins, color="b", histtype="step", label="Best-Fit 3+2 Osc")
    else:
        axs[0, 3].hist(energy_bin_centers, weights=hist[26*3:26*4], bins=energy_bins, color="b", histtype="step")
    for i in range(26):
        axs[0, 3].add_patch(plt.Rectangle((energy_bins[i], hist[26*3+i] - pred_errs[26*3+i]), energy_bins[i+1] - energy_bins[i], 2*pred_errs[26*3+i], hatch="////", lw=0, fill=False, edgecolor="grey"))
    axs[0, 3].errorbar(energy_bin_centers, data_hist[26*3:26*4], yerr=data_errs[26*3:26*4], fmt=".", color="k", label="Data")
    axs[0, 3].set_xlim(0, 2600)
    axs[0, 3].set_title("BNB numuCC PC")
    if no_osc_hist is not None:
        axs[0, 3].hist(energy_bin_centers, weights=no_osc_hist[26*3:26*4], bins=energy_bins, color="r", histtype="step", label="No Osc")
    if no_osc_hist is not None: axs[0, 3].legend()

    axs[1, 0].hist(energy_bin_centers, weights=hist[26*7:26*8], bins=energy_bins, stacked=True, color="b", histtype="step")
    for i in range(26):
        axs[1, 0].add_patch(plt.Rectangle((energy_bins[i], hist[26*7+i] - pred_errs[26*7+i]), energy_bins[i+1] - energy_bins[i], 2*pred_errs[26*7+i], hatch="////", lw=0, fill=False, edgecolor="grey"))
    axs[1, 0].errorbar(energy_bin_centers, data_hist[26*7:26*8], yerr=data_errs[26*7:26*8], fmt=".", color="k", label="Data")
    axs[1, 0].set_xlim(0, 2600)
    axs[1, 0].set_title("NuMI nueCC FC")
    if no_osc_hist is not None:
        axs[1, 0].hist(energy_bin_centers, weights=no_osc_hist[26*7:26*8], bins=energy_bins, color="r", histtype="step")

    axs[1, 1].hist(energy_bin_centers, weights=hist[26*8:26*9], bins=energy_bins, stacked=True, color="b", histtype="step")
    for i in range(26):
        axs[1, 1].add_patch(plt.Rectangle((energy_bins[i], hist[26*8+i] - pred_errs[26*8+i]), energy_bins[i+1] - energy_bins[i], 2*pred_errs[26*8+i], hatch="////", lw=0, fill=False, edgecolor="grey"))
    axs[1, 1].errorbar(energy_bin_centers, data_hist[26*8:26*9], yerr=data_errs[26*8:26*9], fmt=".", color="k", label="Data")
    axs[1, 1].set_xlim(0, 2600)
    axs[1, 1].set_title("NuMI nueCC PC")
    if no_osc_hist is not None:
        axs[1, 1].hist(energy_bin_centers, weights=no_osc_hist[26*8:26*9], bins=energy_bins, color="r", histtype="step")

    axs[1, 2].hist(energy_bin_centers, weights=hist[26*9:26*10], bins=energy_bins, stacked=True, color="b", histtype="step")
    for i in range(26):
        axs[1, 2].add_patch(plt.Rectangle((energy_bins[i], hist[26*9+i] - pred_errs[26*9+i]), energy_bins[i+1] - energy_bins[i], 2*pred_errs[26*9+i], hatch="////", lw=0, fill=False, edgecolor="grey"))
    axs[1, 2].errorbar(energy_bin_centers, data_hist[26*9:26*10], yerr=data_errs[26*9:26*10], fmt=".", color="k", label="Data")
    axs[1, 2].set_xlim(0, 2600)
    axs[1, 2].set_title("NuMI numuCC FC")   
    if no_osc_hist is not None:
        axs[1, 2].hist(energy_bin_centers, weights=no_osc_hist[26*9:26*10], bins=energy_bins, color="r", histtype="step")

    axs[1, 3].hist(energy_bin_centers, weights=hist[26*10:26*11], bins=energy_bins, stacked=True, color="b", histtype="step")
    for i in range(26):
        axs[1, 3].add_patch(plt.Rectangle((energy_bins[i], hist[26*10+i] - pred_errs[26*10+i]), energy_bins[i+1] - energy_bins[i], 2*pred_errs[26*10+i], hatch="////", lw=0, fill=False, edgecolor="grey"))
    axs[1, 3].errorbar(energy_bin_centers, data_hist[26*10:26*11], yerr=data_errs[26*10:26*11], fmt=".", color="k", label="Data")
    axs[1, 3].set_xlim(0, 2600)
    axs[1, 3].set_title("NuMI numuCC PC")
    if no_osc_hist is not None:
        axs[1, 3].hist(energy_bin_centers, weights=no_osc_hist[26*10:26*11], bins=energy_bins, color="r", histtype="step")

    plt.show()


In [ ]:
hist, cov = get_hist_cov(sig_arrs)
print("eigenvalues of cov:", np.linalg.eigvalsh(cov))

cov += np.diag(hist + 0.001) # adding Pearson data stat unceratainty

print("eigenvalues of cov:", np.linalg.eigvalsh(cov))

bnb_nue_start_index = 0
bnb_constr_start_index = 26*2
numi_nue_start_index = 26*7
numi_constr_start_index = 26*7 + 26*2

new_ordering = np.concatenate([
    np.arange(bnb_nue_start_index, bnb_nue_start_index + 26*2),
    np.arange(numi_nue_start_index, numi_nue_start_index + 26*2),
    np.arange(bnb_constr_start_index, bnb_constr_start_index + 26*5),
    np.arange(numi_constr_start_index, numi_constr_start_index + 26*5),
])

new_cov = cov[new_ordering, :][:, new_ordering]
new_data_hist = data_hist[new_ordering]
new_hist = hist[new_ordering]

nue_preds = new_hist[:26*4]
constraining_preds = new_hist[26*4:]

nue_data = new_data_hist[:26*4]
constraining_data = new_data_hist[26*4:]

nue_cov = new_cov[:26*4, :26*4]
constraining_cov = new_cov[26*4:, 26*4:]
mix_cov = new_cov[:26*4, 26*4:]

constrained_nue_preds = nue_preds + mix_cov @ np.linalg.inv(constraining_cov) @ (constraining_data - constraining_preds)
constrained_nue_cov = nue_cov - mix_cov @ np.linalg.inv(constraining_cov) @ mix_cov.T

new_errs = np.sqrt(np.diag(new_cov))
constrained_nue_errs = np.sqrt(np.diag(constrained_nue_cov))


In [ ]:
def plot_constrained_nue_hists(sig_arrs, plot_before_constraint=True, no_osc_nue_hist=None):

    hist, cov = get_hist_cov(sig_arrs)

    cov += np.diag(hist + 0.001) # adding Pearson data stat unceratainty

    # reorder hist and cov to have all four nue channels first 

    bnb_nue_start_index = 0
    bnb_constr_start_index = 26*2
    numi_nue_start_index = 26*7
    numi_constr_start_index = 26*7 + 26*2

    new_ordering = np.concatenate([
        np.arange(bnb_nue_start_index, bnb_nue_start_index + 26*2),
        np.arange(numi_nue_start_index, numi_nue_start_index + 26*2),
        np.arange(bnb_constr_start_index, bnb_constr_start_index + 26*5),
        np.arange(numi_constr_start_index, numi_constr_start_index + 26*5),
    ])

    new_data_hist = data_hist[new_ordering]
    new_data_errs = np.sqrt(new_data_hist)
    new_hist = hist[new_ordering]
    new_cov = cov[new_ordering, :][:, new_ordering]

    nue_preds = new_hist[:26*4]
    constraining_preds = new_hist[26*4:]

    nue_data = new_data_hist[:26*4]
    constraining_data = new_data_hist[26*4:]

    nue_cov = new_cov[:26*4, :26*4]
    constraining_cov = new_cov[26*4:, 26*4:]
    mix_cov = new_cov[:26*4, 26*4:]

    constrained_nue_preds = nue_preds + mix_cov @ np.linalg.inv(constraining_cov) @ (constraining_data - constraining_preds)
    constrained_nue_cov = nue_cov - mix_cov @ np.linalg.inv(constraining_cov) @ mix_cov.T

    new_errs = np.sqrt(np.diag(new_cov))
    constrained_nue_errs = np.sqrt(np.diag(constrained_nue_cov))
    

    fig, axs = plt.subplots(1, 4, figsize=(15, 4))

    if no_osc_nue_hist is not None:
        axs[0].hist(energy_bin_centers, weights=no_osc_nue_hist[:26], bins=energy_bins, color="g", histtype="step")
    if plot_before_constraint:
        axs[0].hist(energy_bin_centers, weights=nue_preds[:26], bins=energy_bins, color="r", histtype="step")
        for i in range(26):
            axs[0].add_patch(plt.Rectangle((energy_bins[i], nue_preds[i] - new_errs[i]), energy_bins[i+1] - energy_bins[i], 2*new_errs[i], hatch="////", lw=0, fill=False, edgecolor="r"))
    axs[0].hist(energy_bin_centers, weights=constrained_nue_preds[:26], bins=energy_bins, color="b", histtype="step")
    for i in range(26):
        axs[0].add_patch(plt.Rectangle((energy_bins[i], constrained_nue_preds[i] - constrained_nue_errs[i]), energy_bins[i+1] - energy_bins[i], 2*constrained_nue_errs[i], hatch=r"\\\\", lw=0, fill=False, edgecolor="b"))
    axs[0].errorbar(energy_bin_centers, nue_data[:26], yerr=new_data_errs[:26], fmt=".", color="k", label="Data")
    axs[0].set_xlim(0, 2600)
    axs[0].set_title("BNB nueCC FC")

    if no_osc_nue_hist is not None:
        axs[1].hist(energy_bin_centers, weights=no_osc_nue_hist[26:26*2], bins=energy_bins, color="g", histtype="step")
    if plot_before_constraint:
        axs[1].hist(energy_bin_centers, weights=nue_preds[26:26*2], bins=energy_bins, color="r", histtype="step")
        for i in range(26):
            axs[1].add_patch(plt.Rectangle((energy_bins[i], nue_preds[26+i] - new_errs[26+i]), energy_bins[i+1] - energy_bins[i], 2*new_errs[26+i], hatch="////", lw=0, fill=False, edgecolor="r"))
    axs[1].hist(energy_bin_centers, weights=constrained_nue_preds[26:26*2], bins=energy_bins, color="b", histtype="step")
    for i in range(26):
        axs[1].add_patch(plt.Rectangle((energy_bins[i], constrained_nue_preds[26+i] - constrained_nue_errs[26+i]), energy_bins[i+1] - energy_bins[i], 2*constrained_nue_errs[26+i], hatch=r"\\\\", lw=0, fill=False, edgecolor="b"))
    axs[1].errorbar(energy_bin_centers, nue_data[26:26*2], yerr=new_data_errs[26:26*2], fmt=".", color="k", label="Data")
    axs[1].set_xlim(0, 2600)
    axs[1].set_title("BNB nueCC PC")

    if no_osc_nue_hist is not None:
        axs[2].hist(energy_bin_centers, weights=no_osc_nue_hist[26*2:26*3], bins=energy_bins, color="g", histtype="step")
    if plot_before_constraint:
        axs[2].hist(energy_bin_centers, weights=nue_preds[26*2:26*3], bins=energy_bins, color="r", histtype="step")
        for i in range(26):
            axs[2].add_patch(plt.Rectangle((energy_bins[i], nue_preds[26*2+i] - new_errs[26*2+i]), energy_bins[i+1] - energy_bins[i], 2*new_errs[26*2+i], hatch="////", lw=0, fill=False, edgecolor="r"))
    axs[2].hist(energy_bin_centers, weights=constrained_nue_preds[26*2:26*3], bins=energy_bins, color="b", histtype="step")
    for i in range(26):
        axs[2].add_patch(plt.Rectangle((energy_bins[i], constrained_nue_preds[26*2+i] - constrained_nue_errs[26*2+i]), energy_bins[i+1] - energy_bins[i], 2*constrained_nue_errs[26*2+i], hatch=r"\\\\", lw=0, fill=False, edgecolor="b"))
    axs[2].errorbar(energy_bin_centers, nue_data[26*2:26*3], yerr=new_data_errs[26*2:26*3], fmt=".", color="k", label="Data")
    axs[2].set_xlim(0, 2600)
    axs[2].set_title("NuMI nueCC FC")

    if no_osc_nue_hist is not None:
        axs[3].hist(energy_bin_centers, weights=no_osc_nue_hist[26*3:26*4], bins=energy_bins, color="g", histtype="step", label="No Osc After Constraint")
    if plot_before_constraint:
        axs[3].hist(energy_bin_centers, weights=nue_preds[26*3:26*4], bins=energy_bins, color="r", histtype="step", label="Before Constraint")
        for i in range(26):
            axs[3].add_patch(plt.Rectangle((energy_bins[i], nue_preds[26*3+i] - new_errs[26*3+i]), energy_bins[i+1] - energy_bins[i], 2*new_errs[26*3+i], hatch="////", lw=0, fill=False, edgecolor="r"))
    axs[3].hist(energy_bin_centers, weights=constrained_nue_preds[26*3:26*4], bins=energy_bins, color="b", histtype="step", label="After Constraint")
    for i in range(26):
        axs[3].add_patch(plt.Rectangle((energy_bins[i], constrained_nue_preds[26*3+i] - constrained_nue_errs[26*3+i]), energy_bins[i+1] - energy_bins[i], 2*constrained_nue_errs[26*3+i], hatch=r"\\\\", lw=0, fill=False, edgecolor="b"))
    axs[3].errorbar(energy_bin_centers, nue_data[26*3:26*4], yerr=new_data_errs[26*3:26*4], fmt=".", color="k", label="Data")
    axs[3].set_xlim(0, 2600)
    axs[3].set_title("NuMI nueCC PC")
    axs[3].legend()

    plt.show()



# 3+2 Oscillation

In [ ]:
def get_3_plus_2_prob(alpha, beta, E, L, osc_params):

    #if alpha == 12 and beta == 12: print("oscillating:", osc_params)

    m4, m5, eta, Ue4, Umu4, Ue5, Umu5 = osc_params

    # all 5x5 complex PMNS matrices have 50 parameters
    # only considering the last two columns for sterile mixings, this is 9 parameters
    # with unitarity constraints, this becomes 7 parameters: theta and delta for each of 14, 24, 34, 15, 25, 35, and 45
    # from https://arxiv.org/pdf/1107.1452, for nue->nue, numu->numu, and nume->nue, we can reduce that to just five parameters: magnitudes of Ue4, Umu4, Ue5, and Umu5, and a phase eta
    # We also still need to require small Ue4, Umu4, Ue5, and Umu5, in order to avoid unitarity violation, but I won't put a strict cut on that here,
    # since it depends on all the measured mixing parameters, which are complicated with large uncertainties.

    # alpha and beta describe the flavors, options: numu, nue, antinue, antinumu
    # m4 and m5 in eV, approximating that m1, m2, and m3 are zero (normal ordering, rather than inverted or perverted)
    # E in GeV, L in km
    # eta is related to the phases of the U elements
    # U elements are the magnitudes of the complex numbers in the extended PMNS matrix

    # E and L are arrays (no if statements on these)

    for flavor in [alpha, beta]:
        assert flavor in [-12, 12, 14, -14], f"Invalid flavor: {flavor}!"
    #assert 0 < m4 < m5, f"Invalid masses: {m4} and {m5}!"
    assert np.min(E) > 0, f"Invalid energy: {E}!"
    assert np.min(L) > 0, f"Invalid length: {L}!"
    for U_elem in [Ue4, Umu4, Ue5, Umu5]:
        if not (0 <= U_elem <= 1):
            pass
            # assert 0 <= U_elem <= 1, f"Invalid U element: {U_elem}!"

    phi_41 = 1.27 * m4**2 * L / E 
    phi_51 = 1.27 * m5**2 * L / E
    phi_54 = 1.27 * (m5**2 - m4**2) * L / E

    if abs(alpha) == 14 and abs(beta) == 12: # numu -> nue
        if alpha == -14: 
            cp_sign = 1
        else:
            cp_sign = -1

        P = 4 * Umu4**2 * Ue4**2 * np.sin(phi_41)**2
        P += 4 * Umu5**2 * Ue5**2 * np.sin(phi_51)**2
        P += 8 * Umu4 * Ue4 * Umu5 * Ue5 * np.sin(phi_41) * np.sin(phi_51) * np.cos(phi_54 + cp_sign * eta)

        return P

    elif alpha == beta: # nu_alpha -> nu_alpha
        if alpha == 12:

            P = 1 - 4 * (1 - Ue4**2 - Ue5**2) * (Ue4**2 * np.sin(phi_41)**2 + Ue5**2 * np.sin(phi_51)**2)
            P -= 4 * Ue4**2 * Ue5**2 * np.sin(phi_54)**2

            return P
        
        elif alpha == 14:
            P = 1 - 4 * (1 - Umu4**2 - Umu5**2) * (Umu4**2 * np.sin(phi_41)**2 + Umu5**2 * np.sin(phi_51)**2)
            P -= 4 * Umu4**2 * Umu5**2 * np.sin(phi_54)**2

            return P

    raise ValueError(f"{alpha} -> {beta} oscillation not implemented!")

def oscillate_arr(arr, alpha, beta, osc_params):
    E = arr[:, 0]
    L = arr[:, 1]
    #non_osc_weight = arr[:, 2]
    #osc_weight = arr[:, 3] # this is the variable we are replacing
    arr[:, 3] = get_3_plus_2_prob(alpha, beta, E, L, osc_params)

def oscillate_arrs(arrs, osc_params):
    
    nue_disapp_arr, numu_disapp_arr, numu_to_nue_arr, antinumu_to_antinue_arr = arrs

    oscillate_arr(nue_disapp_arr, 12, 12, osc_params)
    oscillate_arr(numu_disapp_arr, 14, 14, osc_params)
    oscillate_arr(numu_to_nue_arr, 14, 12, osc_params)
    oscillate_arr(antinumu_to_antinue_arr, -14, -12, osc_params)


In [ ]:
hist, cov = get_hist_cov(sig_arrs)

cov += np.diag(hist + 0.001) # adding Pearson data stat unceratainty

# reorder hist and cov to have all four nue channels first 

bnb_nue_start_index = 0
bnb_constr_start_index = 26*2
numi_nue_start_index = 26*7
numi_constr_start_index = 26*7 + 26*2

new_ordering = np.concatenate([
    np.arange(bnb_nue_start_index, bnb_nue_start_index + 26*2),
    np.arange(numi_nue_start_index, numi_nue_start_index + 26*2),
    np.arange(bnb_constr_start_index, bnb_constr_start_index + 26*5),
    np.arange(numi_constr_start_index, numi_constr_start_index + 26*5),
])

new_data_hist = data_hist[new_ordering]
new_data_errs = np.sqrt(new_data_hist)
new_hist = hist[new_ordering]
new_cov = cov[new_ordering, :][:, new_ordering]

nue_preds = new_hist[:26*4]
constraining_preds = new_hist[26*4:]

nue_data = new_data_hist[:26*4]
constraining_data = new_data_hist[26*4:]

nue_cov = new_cov[:26*4, :26*4]
constraining_cov = new_cov[26*4:, 26*4:]
mix_cov = new_cov[:26*4, 26*4:]

constrained_nue_preds = nue_preds + mix_cov @ np.linalg.inv(constraining_cov) @ (constraining_data - constraining_preds)
constrained_nue_cov = nue_cov - mix_cov @ np.linalg.inv(constraining_cov) @ mix_cov.T

new_errs = np.sqrt(np.diag(new_cov))
constrained_nue_errs = np.sqrt(np.diag(constrained_nue_cov))

no_osc_hist = hist
no_osc_constrained_hist = constrained_nue_preds


In [ ]:
m4 = 1
m5 = 1000
eta = 0
Ue4 = 0
Umu4 = 0
Ue5 = 0
Umu5 = 0

osc_params = [m4, m5, eta, Ue4, Umu4, Ue5, Umu5]

oscillate_arrs(sig_arrs, osc_params)

plot_nue_numu_hists(sig_arrs)
plot_constrained_nue_hists(sig_arrs)


# Minimizing 3+1 chi2

In [ ]:
def get_chi2(three_plus_one_osc_params):

    m4, Ue4, Umu4, = three_plus_one_osc_params
    m5, Ue5, Umu5, eta = 1e9, 0, 0, 0

    osc_params = [m4, m5, eta, Ue4, Umu4, Ue5, Umu5]

    oscillate_arrs(sig_arrs, osc_params)
    hist, cov = get_hist_cov(sig_arrs)

    if not (0 <= Ue4 <= 1 and 0 <= Umu4 <= 1):
        return 1e9 # high value for minimizer
    if m4 < 0:
        return 1e9 # high value for minimizer
    
    cov += np.diag(hist + 0.001)
    diffs = data_hist - hist
    chi2 = diffs @ np.linalg.inv(cov) @ diffs
    return chi2

from scipy.optimize import minimize

three_plus_one_start_params = [[1, 0, 0]]
m4_starting_possibilities = np.logspace(np.log10(0.01), np.log10(10), 100)
Ue4_starting_possibilities = np.logspace(np.log10(0.01), np.log10(1), 100)
Umu4_starting_possibilities = np.logspace(np.log10(0.01), np.log10(1), 100)

all_bf_chi2s = []
all_bf_params = []
all_num_evals = []

num_starts = 10
for i in range(num_starts):

    m4 = np.random.choice(m4_starting_possibilities)
    Ue4 = np.random.choice(Ue4_starting_possibilities)
    Umu4 = np.random.choice(Umu4_starting_possibilities)

    if i == 0: # trying no-osc starting point
        m4 = 1
        Ue4 = 0
        Umu4 = 0

    if i == 1: # trying best known 3+1 point
        with open("best_fit_3_1_osc_params.pkl", "rb") as f:
            best_fit_3_1_osc_params = pickle.load(f)
        m4, m5, eta, Ue4, Umu4, Ue5, Umu5 = best_fit_3_1_osc_params
    
    x0 = [m4, Ue4, Umu4]

    print(f"starting guess: chi2: {get_chi2(x0)}, params: {x0}")

    #res = minimize(get_chi2, x0, method="Nelder-Mead")
    #res = minimize(get_chi2, x0, method="L-BFGS-B")
    res = minimize(get_chi2, x0, method="Powell")
    bf_chi2 = res.fun
    bf_params = res.x
    num_evals = res.nfev
    print(f"    best fit chi2: {bf_chi2}, params: {bf_params}, num evals: {num_evals}")

    all_bf_chi2s.append(bf_chi2)
    all_bf_params.append(bf_params)
    all_num_evals.append(num_evals)


In [ ]:
best_fit_index = np.argmin(all_bf_chi2s)
bf_params = all_bf_params[best_fit_index]

m4, Ue4, Umu4, = bf_params
m5, Ue5, Umu5, eta = 1e9, 0, 0, 0

best_fit_3_1_osc_params = [m4, m5, eta, Ue4, Umu4, Ue5, Umu5]

with open("best_fit_3_1_osc_params.pkl", "wb") as f:
    pickle.dump(best_fit_3_1_osc_params, f)

print("best fit m4:", m4)
print("best fit Ue4:", Ue4)
print("best fit Umu4:", Umu4)

oscillate_arrs(sig_arrs, best_fit_3_1_osc_params)

plot_nue_numu_hists(sig_arrs, no_osc_hist=no_osc_hist)
plot_constrained_nue_hists(sig_arrs, no_osc_nue_hist=no_osc_constrained_hist)
plot_constrained_nue_hists(sig_arrs, plot_before_constraint=False, no_osc_nue_hist=no_osc_constrained_hist)


# Minimizing 3+2 chi2

In [ ]:
def get_chi2(osc_params):
    oscillate_arrs(sig_arrs, osc_params)
    hist, cov = get_hist_cov(sig_arrs)

    m4, m5, eta, Ue4, Umu4, Ue5, Umu5 = osc_params

    if not (0 <= Ue4 <= 1 and 0 <= Umu4 <= 1 and 0 <= Ue5 <= 1 and 0 <= Umu5 <= 1):
        return 1e9
    if m4 < 0 or m5 <= m4:
        return 1e9
    if Ue4**2 + Ue5**2 > 1 or Umu4**2 + Umu5**2 > 1:
        return 1e9 # high value for minimizer
    
    cov += np.diag(hist + 0.001)
    diffs = data_hist - hist
    chi2 = diffs @ np.linalg.inv(cov) @ diffs
    return chi2

from scipy.optimize import minimize

three_plus_one_start_params = [[1, 5, 0, 0, 0, 0, 0]]
m4_starting_possibilities = np.logspace(np.log10(0.01), np.log10(100), 100)
m5_starting_possibilities = np.logspace(np.log10(0.01), np.log10(1000), 100)
eta_starting_possibilities = np.linspace(0, 2*np.pi, 100)
Ue4_starting_possibilities = np.logspace(np.log10(0.01), np.log10(1), 100)
Umu4_starting_possibilities = np.logspace(np.log10(0.01), np.log10(1), 100)
Ue5_starting_possibilities = np.logspace(np.log10(0.01), np.log10(1), 100)
Umu5_starting_possibilities = np.logspace(np.log10(0.01), np.log10(1), 100)

all_bf_chi2s = []
all_bf_params = []
all_num_evals = []

num_starts = 10
for i in range(num_starts):

    made_choice = False
    while (not made_choice or get_chi2([m4, m5, eta, Ue4, Umu4, Ue5, Umu5]) > 1e6):
        m4 = np.random.choice(m4_starting_possibilities)
        m5 = np.random.choice(m5_starting_possibilities)
        eta = np.random.choice(eta_starting_possibilities)
        Ue4 = np.random.choice(Ue4_starting_possibilities)
        Umu4 = np.random.choice(Umu4_starting_possibilities)
        Ue5 = np.random.choice(Ue5_starting_possibilities)
        Umu5 = np.random.choice(Umu5_starting_possibilities)
        made_choice = True

    if i == 0: # trying no-osc starting point
        m4 = 1
        m5 = 5
        eta = 0
        Ue4 = 0
        Umu4 = 0
        Ue5 = 0
        Umu5 = 0

    if i == 1: # trying best known 3+2 point
        with open("best_fit_3_2_osc_params.pkl", "rb") as f:
            best_fit_3_2_osc_params = pickle.load(f)
        m4, m5, eta, Ue4, Umu4, Ue5, Umu5 = best_fit_3_2_osc_params

    x0 = [m4, m5, eta, Ue4, Umu4, Ue5, Umu5]

    print(f"starting guess: chi2: {get_chi2(x0)}, params: {x0}")

    #res = minimize(get_chi2, x0, method="Nelder-Mead")
    #res = minimize(get_chi2, x0, method="L-BFGS-B")
    res = minimize(get_chi2, x0, method="Powell")
    bf_chi2 = res.fun
    bf_params = res.x
    num_evals = res.nfev
    print(f"    best fit chi2: {bf_chi2}, params: {bf_params}, num evals: {num_evals}")

    all_bf_chi2s.append(bf_chi2)
    all_bf_params.append(bf_params)
    all_num_evals.append(num_evals)


In [ ]:
best_fit_index = np.argmin(all_bf_chi2s)
best_fit_3_2_osc_params = all_bf_params[best_fit_index]

with open("best_fit_3_2_osc_params.pkl", "wb") as f:
    pickle.dump(best_fit_3_2_osc_params, f)


m4, m5, eta, Ue4, Umu4, Ue5, Umu5 = best_fit_3_2_osc_params

print("best fit m4:", m4)
print("best fit m5:", m5)
print("best fit eta:", eta)
print("best fit Ue4:", Ue4)
print("best fit Umu4:", Umu4)
print("best fit Ue5:", Ue5)
print("best fit Umu5:", Umu5)

oscillate_arrs(sig_arrs, best_fit_3_2_osc_params)

plot_nue_numu_hists(sig_arrs, no_osc_hist=no_osc_hist)
plot_constrained_nue_hists(sig_arrs, no_osc_nue_hist=no_osc_constrained_hist)
plot_constrained_nue_hists(sig_arrs, plot_before_constraint=False, no_osc_nue_hist=no_osc_constrained_hist)


# Looking at L/E

In [ ]:
bnb_nuesel_nue_df = all_sig_df.query("beam=='BNB' and (sel=='nueCC_FC' or sel=='nueCC_PC') and filetype=='intrinsic'")
bnb_nuesel_nue_l_e = bnb_nuesel_nue_df["e2e_baseline"].to_numpy() / bnb_nuesel_nue_df["e2e_Etrue"].to_numpy()
bnb_nuesel_oscnumu_df = all_sig_df.query("beam=='BNB' and (sel=='nueCC_FC' or sel=='nueCC_PC') and filetype=='appnue'")
bnb_nuesel_oscnumu_l_e = bnb_nuesel_oscnumu_df["e2e_baseline"].to_numpy() / bnb_nuesel_oscnumu_df["e2e_Etrue"].to_numpy()
numi_nuesel_nue_df = all_sig_df.query("beam=='NuMI' and (sel=='nueCC_FC' or sel=='nueCC_PC') and filetype=='intrinsic'")
numi_nuesel_nue_l_e = numi_nuesel_nue_df["e2e_baseline"].to_numpy() / numi_nuesel_nue_df["e2e_Etrue"].to_numpy()
numi_nuesel_oscnumu_df = all_sig_df.query("beam=='NuMI' and (sel=='nueCC_FC' or sel=='nueCC_PC') and filetype=='appnue'")
numi_nuesel_oscnumu_l_e = numi_nuesel_oscnumu_df["e2e_baseline"].to_numpy() / numi_nuesel_oscnumu_df["e2e_Etrue"].to_numpy()


bins = np.logspace(np.log10(0.03), np.log10(10), 100)
fig, axs = plt.subplots(3, 1, figsize=(10, 8))
axs[0].hist(bnb_nuesel_nue_l_e, bins=bins, histtype="step", label="BNB nue")
axs[0].hist(bnb_nuesel_oscnumu_l_e, bins=bins, histtype="step", label="BNB numu")
axs[0].hist(numi_nuesel_nue_l_e, bins=bins, histtype="step", label="NuMI nue")
axs[0].hist(numi_nuesel_oscnumu_l_e, bins=bins, histtype="step", label="NuMI numu")
axs[0].legend(loc="upper right")
axs[0].set_xlim(bins[0], bins[-1])
axs[0].set_xscale("log")
axs[0].set_ylabel("Counts")
axs[0].set_yscale("log")
axs[0].set_title(r"For events selected as $\nu_e$CC FC or PC""\n"r"after oscillating to true $\nu_e$")
l_e_arr = np.linspace(bins[0], bins[-1], 100000)
ones_arr = np.ones(len(l_e_arr))
nue_disapp_probs = get_3_plus_2_prob(12, 12, ones_arr, l_e_arr, best_fit_3_2_osc_params)
numu_disapp_probs = get_3_plus_2_prob(14, 14, ones_arr, l_e_arr, best_fit_3_2_osc_params)
numu_to_nue_probs = get_3_plus_2_prob(14, 12, ones_arr, l_e_arr, best_fit_3_2_osc_params)
antinumu_to_antinue_probs = get_3_plus_2_prob(-14, -12, ones_arr, l_e_arr, best_fit_3_2_osc_params)
axs[1].plot(l_e_arr, 1 - nue_disapp_probs, label=r"3+2 1-P$(\nu_e\rightarrow\nu_e$)")
axs[1].plot(l_e_arr, 1 - numu_disapp_probs, label=r"3+2 1-P$(\nu_\mu\rightarrow\nu_\mu$)")
axs[1].plot(l_e_arr, numu_to_nue_probs, label=r"3+2 $P(\nu_\mu\rightarrow\nu_e$)")
axs[1].plot(l_e_arr, antinumu_to_antinue_probs, label=r"3+2 $P(\bar{\nu}_\mu\rightarrow\bar{\nu}_e$)")
axs[1].set_ylabel("Probability")
axs[1].set_xlim(bins[0], bins[-1])
axs[1].set_xscale("log")
axs[1].legend(loc="upper right")
nue_disapp_probs = get_3_plus_2_prob(12, 12, ones_arr, l_e_arr, best_fit_3_1_osc_params)
numu_disapp_probs = get_3_plus_2_prob(14, 14, ones_arr, l_e_arr, best_fit_3_1_osc_params)
numu_to_nue_probs = get_3_plus_2_prob(14, 12, ones_arr, l_e_arr, best_fit_3_1_osc_params)
antinumu_to_antinue_probs = get_3_plus_2_prob(-14, -12, ones_arr, l_e_arr, best_fit_3_1_osc_params)
axs[2].plot(l_e_arr, 1 - nue_disapp_probs, label=r"3+1 1-P$(\nu_e\rightarrow\nu_e$)")
axs[2].plot(l_e_arr, 1 - numu_disapp_probs, label=r"3+1 1-P$(\nu_\mu\rightarrow\nu_\mu$)")
axs[2].plot(l_e_arr, numu_to_nue_probs, label=r"3+1 $P(\nu_\mu\rightarrow\nu_e$)")
axs[2].plot(l_e_arr, antinumu_to_antinue_probs, label=r"3+1 $P(\bar{\nu}_\mu\rightarrow\bar{\nu}_e$)")
axs[2].legend(loc="upper right")
axs[2].set_ylabel("Probability")
axs[2].set_xlim(bins[0], bins[-1])
axs[2].set_xscale("log")
axs[2].set_xlabel("L/E (km/GeV)")
plt.show()

bins = np.linspace(0.0000001, 5, 100)
fig, axs = plt.subplots(3, 1, figsize=(10, 8))
axs[0].hist(bnb_nuesel_nue_l_e, bins=bins, histtype="step", label="BNB nue")
axs[0].hist(bnb_nuesel_oscnumu_l_e, bins=bins, histtype="step", label="BNB numu")
axs[0].hist(numi_nuesel_nue_l_e, bins=bins, histtype="step", label="NuMI nue")
axs[0].hist(numi_nuesel_oscnumu_l_e, bins=bins, histtype="step", label="NuMI numu")
axs[0].legend(loc="upper right")
axs[0].set_xlim(bins[0], bins[-1])
axs[0].set_ylabel("Counts")
axs[0].set_yscale("log")
axs[0].set_title(r"For events selected as $\nu_e$CC FC or PC""\n"r"after oscillating to true $\nu_e$")
l_e_arr = np.linspace(bins[0], bins[-1], 1_000_000)
ones_arr = np.ones(len(l_e_arr))
nue_disapp_probs = get_3_plus_2_prob(12, 12, ones_arr, l_e_arr, best_fit_3_2_osc_params)
numu_disapp_probs = get_3_plus_2_prob(14, 14, ones_arr, l_e_arr, best_fit_3_2_osc_params)
numu_to_nue_probs = get_3_plus_2_prob(14, 12, ones_arr, l_e_arr, best_fit_3_2_osc_params)
antinumu_to_antinue_probs = get_3_plus_2_prob(-14, -12, ones_arr, l_e_arr, best_fit_3_2_osc_params)
axs[1].plot(l_e_arr, 1 - nue_disapp_probs, label=r"3+2 1-P$(\nu_e\rightarrow\nu_e$)")
axs[1].plot(l_e_arr, 1 - numu_disapp_probs, label=r"3+2 1-P$(\nu_\mu\rightarrow\nu_\mu$)")
axs[1].plot(l_e_arr, numu_to_nue_probs, label=r"3+2 $P(\nu_\mu\rightarrow\nu_e$)")
axs[1].plot(l_e_arr, antinumu_to_antinue_probs, label=r"3+2 $P(\bar{\nu}_\mu\rightarrow\bar{\nu}_e$)")
axs[1].set_ylabel("Probability")
axs[1].set_xlim(bins[0], bins[-1])
axs[1].legend(loc="upper right")
nue_disapp_probs = get_3_plus_2_prob(12, 12, ones_arr, l_e_arr, best_fit_3_1_osc_params)
numu_disapp_probs = get_3_plus_2_prob(14, 14, ones_arr, l_e_arr, best_fit_3_1_osc_params)
numu_to_nue_probs = get_3_plus_2_prob(14, 12, ones_arr, l_e_arr, best_fit_3_1_osc_params)
antinumu_to_antinue_probs = get_3_plus_2_prob(-14, -12, ones_arr, l_e_arr, best_fit_3_1_osc_params)
axs[2].plot(l_e_arr, 1 - nue_disapp_probs, label=r"3+1 1-P$(\nu_e\rightarrow\nu_e$)")
axs[2].plot(l_e_arr, 1 - numu_disapp_probs, label=r"3+1 1-P$(\nu_\mu\rightarrow\nu_\mu$)")
axs[2].plot(l_e_arr, numu_to_nue_probs, label=r"3+1 $P(\nu_\mu\rightarrow\nu_e$)")
axs[2].plot(l_e_arr, antinumu_to_antinue_probs, label=r"3+1 $P(\bar{\nu}_\mu\rightarrow\bar{\nu}_e$)")
axs[2].legend(loc="upper right")
axs[2].set_ylabel("Probability")
axs[2].set_xlim(bins[0], bins[-1])
axs[2].set_xlabel("L/E (km/GeV)")
plt.show()
